<a href="https://colab.research.google.com/github/ali-rn/project-3/blob/main/nnunet_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/neheller/kits23

Cloning into 'kits23'...
remote: Enumerating objects: 15605, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 15605 (delta 36), reused 63 (delta 30), pack-reused 15526 (from 1)
Receiving objects: 100% (15605/15605), 493.94 MiB | 46.48 MiB/s, done.
Resolving deltas: 100% (14005/14005), done.
Updating files: 100% (5720/5720), done.


In [6]:
%cd kits23
!pip3 install .
!kits23_download_data

/content/kits23
Processing /content/kits23
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/deepmind/surface-distance.git to /tmp/pip-install-gb1jhuzy/surface-distance_d517803271784038bc2b4a41690f4278
  Running command git clone --filter=blob:none --quiet https://github.com/deepmind/surface-distance.git /tmp/pip-install-gb1jhuzy/surface-distance_d517803271784038bc2b4a41690f4278
  Resolved https://github.com/deepmind/surface-distance.git to commit 1f805cea446805e559badc80dd27f34175e4173d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.3 MB/s eta 0:00:00
  Created wheel for kits23: filename=kits23-0.1.4-py3-none-any.whl size=26451 sha256=ad12de3cb1324fb86f7e3635337073e384a9c1f6aee114c599a78456577ac590
  Stored in direct


Found 489 cases to download

Dowloading case_00588...: 100% 489/489 [09:54<00:00,  1.22s/it]


In [7]:
import os
from glob import glob

example_case = "/content/kits23/dataset/case_00000"
print("Contents of case_00000:")
print(os.listdir(example_case))

Contents of case_00000:
['instances', 'segmentation.nii.gz']


In [17]:
import os
import json
import nibabel as nib
from glob import glob
from shutil import copyfile

kits23_dir = "/content/kits23/dataset"
nnunet_dir = "/content/nnUNet_raw/Dataset502_KiTS23"
imagesTr_dir = os.path.join(nnunet_dir, "imagesTr")
labelsTr_dir = os.path.join(nnunet_dir, "labelsTr")

os.makedirs(imagesTr_dir, exist_ok=True)
os.makedirs(labelsTr_dir, exist_ok=True)

case_dirs = sorted(glob(os.path.join(kits23_dir, "case_*")))
training_data = []
skipped = 0
max_cases = 20 # or even 20 for debugging

for idx, case_path in enumerate(case_dirs[:max_cases]):
    case_id = os.path.basename(case_path)
    instance_files = glob(os.path.join(case_path, "instances", "*.nii.gz"))
    if not instance_files:
        skipped += 1
        continue

    image_path = instance_files[0]
    label_path = os.path.join(case_path, "segmentation.nii.gz")
    if not os.path.exists(label_path):
        skipped += 1
        continue

    img_out = f"case_{idx:05d}_0000.nii.gz"
    lbl_out = f"case_{idx:05d}.nii.gz"
    copyfile(image_path, os.path.join(imagesTr_dir, img_out))
    copyfile(label_path, os.path.join(labelsTr_dir, lbl_out))

    training_data.append({
        "image": f"imagesTr/{img_out}",
        "label": f"labelsTr/{lbl_out}"
    })

# FIX: reversed keys -> correct for nnU-Net v2
dataset_json = {
    "channel_names": {"0": "CT"},
    "labels": {
        "background": 0,
        "kidney": 1,
        "tumor": 2,
        "cyst": 3
    },
    "numTraining": len(training_data),
    "file_ending": ".nii.gz"
}

with open(os.path.join(nnunet_dir, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=4)

print(f"Created dataset.json with {len(training_data)} cases")
print(f"Skipped {skipped} cases")

Created dataset.json with 20 cases
Skipped 0 cases


In [18]:
# cleanup_excess_cases.py

import os
from glob import glob

nnunet_dir = "/content/nnUNet_raw/Dataset502_KiTS23"
imagesTr_dir = os.path.join(nnunet_dir, "imagesTr")
labelsTr_dir = os.path.join(nnunet_dir, "labelsTr")

keep_images = set()
keep_labels = set()

for i in range(489):
    keep_images.add(f"case_{i:05d}_0000.nii.gz")
    keep_labels.add(f"case_{i:05d}.nii.gz")

def remove_unlisted(folder, keep_set):
    for file_path in glob(os.path.join(folder, "*.nii.gz")):
        fname = os.path.basename(file_path)
        if fname not in keep_set:
            os.remove(file_path)
            print(f"Removed extra file: {fname}")

remove_unlisted(imagesTr_dir, keep_images)
remove_unlisted(labelsTr_dir, keep_labels)

print("Cleanup complete. Only expected training files remain.")

Cleanup complete. Only expected training files remain.


In [12]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git

Cloning into 'nnUNet'...
remote: Enumerating objects: 13673, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 13673 (delta 1), reused 0 (delta 0), pack-reused 13669 (from 3)
Receiving objects: 100% (13673/13673), 8.04 MiB | 32.56 MiB/s, done.
Resolving deltas: 100% (10475/10475), done.


In [13]:
# Step 3: Install nnUNet
%cd nnUNet
!pip install -e .
%cd ..

/content/nnUNet
Obtaining file:///content/nnUNet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 M

/content


In [19]:
import os
os.environ["nnUNet_raw"] = "/content/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/nnUNet_results"

os.makedirs(os.environ["nnUNet_raw"], exist_ok=True)
os.makedirs(os.environ["nnUNet_preprocessed"], exist_ok=True)
os.makedirs(os.environ["nnUNet_results"], exist_ok=True)

In [20]:
import os

img_path = "/content/nnUNet_raw/Dataset502_KiTS23/imagesTr"
lbl_path = "/content/nnUNet_raw/Dataset502_KiTS23/labelsTr"

images = sorted(os.listdir(img_path))
labels = sorted(os.listdir(lbl_path))

print(f"imagesTr count: {len(images)}")
print(f"labelsTr count: {len(labels)}")
print("Example image:", images[:1])
print("Example label:", labels[:1])

imagesTr count: 20
labelsTr count: 20
Example image: ['case_00000_0000.nii.gz']
Example label: ['case_00000.nii.gz']


In [12]:
import os
import shutil

src_img = "/content/nnUNet_raw/Dataset502_KiTS23/imagesTr"
src_lbl = "/content/nnUNet_raw/Dataset502_KiTS23/labelsTr"

keep = 20  # small fast subset
for idx, f in enumerate(sorted(os.listdir(src_img))):
    if idx >= keep:
        os.remove(os.path.join(src_img, f))
for idx, f in enumerate(sorted(os.listdir(src_lbl))):
    if idx >= keep:
        os.remove(os.path.join(src_lbl, f))

print("✅ Reduced to 20 cases")

✅ Reduced to 20 cases


In [21]:
!nvidia-smi

Mon Apr 21 11:42:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nnUNetv2_plan_and_preprocess -d 502 -c 2d -np 2 --verify_dataset_integrity --clean

Fingerprint extraction...
Dataset502_KiTS23
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


In [17]:
# training
!nnUNetv2_train 3d_fullres 502 0 \
  --npz \
  --c \
  --max_num_epochs=30 \
  --disable_postprocessing_on_folds \
  --batch_size=1 \
  --patch_size 64 64 64

Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_train", line 5, in <module>
    from nnunetv2.run.run_training import run_training_entry
  File "/content/kits23/nnUNet/nnunetv2/run/run_training.py", line 12, in <module>
    from nnunetv2.run.load_pretrained_weights import load_pretrained_weights
  File "/content/kits23/nnUNet/nnunetv2/run/load_pretrained_weights.py", line 2, in <module>
    from torch._dynamo import OptimizedModule
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/__init__.py", line 3, in <module>
    from . import convert_frame, eval_frame, resume_execution
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/convert_frame.py", line 33, in <module>
    from torch._dynamo.symbolic_convert import TensorifyState
  File "/usr/local/lib/python3.11/dist-packages/torch/_dynamo/symbolic_convert.py", line 27, in <module>
    from torch._dynamo.exc import TensorifyScalarRestartAnalysis
  File "/usr/local/lib/python3.11/dist-packages/torch

In [ ]:
#visualization
